# Word2Vec

Word2Vec是一种基于神经网络的embedding模型，由谷歌的研究人员Tomas Mikolov等在2013年提出。Word2Vec的作者认为之前的文本向量化方法很少考虑词与词之间的意义关联，所以效果不佳。因此，Word2Vec将单词映射到低维向量空间中，使得相似词在向量空间中也保持相近的距离。

<img src="images/img_3.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 500px;">

<img src="images/img_7.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 500px;">

Word2Vec主要由两种模型架构组成：**Skip-Gram**和**Continuous Bag of Words（CBOW）**。

Skip-Gram通过当前词预测其上下文词。给定一个中心词，模型会预测一个固定长度的上下文窗口（前后若干词）中的所有词。训练过程中，模型会学习到每个词的向量表示，使得能够更好地预测这些上下文词。与Skip-Gram相反，CBOW的目标是通过上下文词预测当前词。给定一个上下文窗口中的所有词，模型会预测这个窗口的中心词。

Word2Vec模型的核心思想是通过训练神经网络，使得单词与其上下文之间的关系可以在向量空间中被有效地表示。Word2Vec的输入层是一个one-hot向量（one-hot vector），长度为词汇表大小（$V$）。紧接着是一个投影层，由输入层经过一个权重矩阵 $W$（维度为$V \times N$，$N$为嵌入向量的维度），投影到 $N$ 维向量空间中。投影层的输出通过另一个权重矩阵 $W'$（维度为$N \times V$），映射回一个词汇表大小的向量，此为输出层。最后经过一个Softmax层得到每个词的概率。在训练过程中，Word2Vec模型通过最大化目标函数（取决于使用Skip-Gram还是CBOW，以及采用的优化函数的方法）来更新权重矩阵，从而使得embedding能够捕捉到词汇的语义信息。

> 从监督学习的角度来说，word2vec 本质上是一个基于神经网络的多分类问题，当输出词语非常多时， 我们则需要一些像层级 Softmax 和负采样之类的 trick 来加速训练。但从自然语言处理的角度来说， word2vec 关注的并不是神经网络模型本身，而是训练之后得到的词汇的向量化表征。 这种表征使得最后的词向量维度要远远小于词汇表大小，所以 word2vec 从本质上来说是一种降维操作。 我们把数以万计的词汇从高维空间中降维到低维空间中，大大方便了后续的 NLP 分析任务。


## 优缺点

优点：
● 由于采用了浅层神经网络，训练速度较快，适合处理大规模数据。
● 能够捕捉到词与词之间的语义关系，如词性、同义词等。
● 可以用于多种自然语言处理任务，如文本分类、情感分析、机器翻译等。

缺点：
● 每个词只有一个固定的向量表示，无法处理多义词的不同语义。
● 无法考虑词在不同上下文中的语义变化。

## 应用场景
● 信息检索
● 文本分类
● 推荐系统，如根据用户的评论调整推荐的产品


**在Skip-Gram里面，每个词在作为中心词的时候，实际上是 1个学生 VS K个老师，K个老师（周围词）都会对学生（中心词）进行“专业”的训练，这样学生（中心词）的“能力”（向量结果）相对就会扎实（准确）一些，但是这样肯定会使用更长的时间。CBOW是 1个老师 VS K个学生，K个学生（周围词）都会从老师（中心词）那里学习知识，但是老师（中心词）是一视同仁的，教给大家的一样的知识。所以，一般来说 CBOW比Skip-Gram训练速度快，训练过程更加稳定，原因是CBOW使用上下文的方式进行训练，每个训练step会见到更多样本。而在生僻字（出现频率低的字）处理上，skip-gram比CBOW效果更好，学习的词向量更细致，原因就如上面分析：  CBOW 是公共课，Skip-gram 是私教 。**


## CBOW
CBOW 模型的应用场景是要根据上下文预测中间的词，所以输入便是上下文词，当然原始的单词是无法作为输入的， 这里的输入仍然是每个词汇的 One-Hot 向量，输出 Y 为给定词汇表中每个词作为目标词的概率。

CBOW 模型结构是一种普通的神经网络结构。主要包括输入层、中间隐藏层、最后的输出层。 以输入、输出样本 $(Context(w),w)$ 为例对 CBOW 模型的三个网络层进行简单说明， 其中假设 Context(w)由 $w$ 前后各 $c$个词构成。
  - **输入层**: 包含 $Context(w)$ 中 $2c$ 个词的词向量 $v(Context(w)1),v(Context(w)2),⋯,v(Context(w)2c)\in{R^m}$。 这里， $m$ 的含义同上表示词向量的长度
  - **投影层**: 将输入层的 $2c$ 个向量做求和累加，即 $x_w=\sum_{i=1}^{2c}v(Context(w)_i)\in{R^m}$
  - **输出层**: 输出层对应一颗二叉树，它是以语料中出现过的词当叶子节点，以各词在语料中出现的次数当权值构造出来的 Huffman 树。在这棵 Huffman 树中， 叶子节点共 $N(=|D|)$ 个，分别对应词典 $D$ 中的词，非叶子节点 $N−1$ 个。

普通的基于神经网络的语言模型输出层一般就是利用 softmax 函数进行归一化计算，这种直接 softmax 的做法主要问题在于计算速度， 尤其是我们采用了一个较大的词汇表的时候，对大的词汇表做求和运算，softmax 的分运算会非常慢，直接影响到了模型性能。

可以看到，上面提到的取消隐藏层，投影层求和平均都可以一定程度上减少计算量，但输出层的数量在那里， 比如语料库有 500W 个词，那么隐藏层就要对 500W 个神经元进行全连接计算，这依然需要庞大的计算量。 word2vec 算法又在这里进行了训练优化.

除了层级 softmax 输出之外，还有一种叫做负采样的训练 trick

目标函数：
$$
J=\sum_{w \in corpus} P(w|context(w))
$$

CBOW 在 NNLM 基础上有以下几点创新

1. 取消了隐藏层，减少了计算量
2. 采用上下文划窗而不是前文划窗，即用上下文的词来预测当前词
3. 投影层不再使用各向量拼接的方式，而是简单的求和平均

<img src="images/img_4.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 400px;">

## Skip-Gram

Skip-gram 模型的应用场景是要根据中间词预测上下文词，所以输入 $X$ 是任意单词， 输出 $Y$为给定词汇表中每个词作为上下文词的概率。

Skip-gram 模型与 CBOW 模型翻转，也是也是一种普通的神经网络结构， 同样也包括输入层、中间隐藏层和最后的输出层。继续以输入输出样本 $(Context(w)，w)$
 为例 对 Skip-gram 模型的三个网络层进行简单说明， 其中假设 $Context(w)$ 由 $w$ 前后各 $c$ 个词构成。数学细节如下:
- **输入层**: 只含当前样本的中心词 $w$ 的词向量 $v(w)∈Rm$
- **投影层**: 这是个恒等投影，把 $v(w)$ 投影到 $v(w)$，因此，这个投影层其实是多余的。 这里之所以保留投影层主要是方便和 CBOW 模型的网络结构做对比
- **输出层**: 和 CBOW 模型一样，输出层也是一棵 Huffman 树

<img src="images/img_5.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 400px;">

<img src="images/img_6.png" alt="Image" style="display: block; margin-left: auto; margin-right: auto; width: 400px;">

In [3]:
from gensim.models.word2vec import Word2Vec
import gensim

# gensim.__version__

# model = Word2Vec(sentences=topics_list,iter=5, size=128,window=5,min_count=0,workers=10,sg=1,hs=1,negative=1,seed=128,compute_loss=True)

# sentences：训练模型的语料，是一个可迭代的序列
# corpus_file：表示从文件中加载数据，和sentences互斥
# vector_size：word的维度，默认为100，通常取64、128、256等
# window：滑动窗口的大小（词向量上下文最大距离），默认值为5。window越大，则和某一词较远的词也会产生上下文关系。在实际使用中，可以根据实际的需求来动态调整这个window的大小。如果是小语料则这个值可以设的更小。对于一般的语料这个值推荐在[5,10]之间。
# min_count：word次数小于该值被忽略掉（词频），默认值为5。如果是小语料，可以调低这个值。
# seed：用于随机数发生器
# workers：使用多少线程进行模型训练，默认为3
# min_alpha=0.0001：支持的最小迭代率，默认为0.0001
# sg：1 表示 Skip-gram，0 表示 CBOW，默认为0
# hs：1 表示 hierarchical softmax ，0 且 negative 参数不为0 的话 negative sampling 会被启用，默认为0
# negative：0 表示不采用，1 表示采用，建议值在 5-20 表示噪音词的个数，默认为5
# iter：迭代次数，默认为5
# compute_loss：是否计算损失值，默认值为False

# model.wv.key_to_index 词表到索引
# model.wv.index_to_key 所有的词
# model.wv.vectors 每个词的词向量
# model.wv.vector_size 词向量的维度
# model.wv.get_vector(word) 返回给定单词的词向量
# model.wv.most_similar(word, topn=10) 返回最相似的10个词
# model.wv.similarity(word1, word2) 返回两个词之间的余弦相似度

In [20]:
import jieba
import re
import os
from gensim.models import word2vec
import time

In [21]:
def read_stop(stop_words_path):
    """
    读取停用词
    :return:
    """
    # 读取停用词
    stop_words = []
    with open(stop_words_path, "r", encoding="utf-8") as f_reader:
        for line in f_reader:
            line = line.replace("\r", "").replace("\n", "").strip()
            stop_words.append(line)
    # print(len(stop_words))
    stop_words = set(stop_words)
    # print(len(stop_words))
    return stop_words

In [22]:
def data_process(stop_words, data_path, split_data_path):
    """
    数据预处理
    :return:
    """

    # 文本预处理
    sentecnces = []
    rules = u"[\u4e00-\u9fa5]+"
    pattern = re.compile(rules)
    f_writer = open(split_data_path, "w", encoding="utf-8")

    with open(data_path, "r", encoding="utf-8") as f_reader:
        for line in f_reader:
            line = line.replace("\r", "").replace("\n", "").strip()
            if line == "" or line is None:
                continue
            line = " ".join(jieba.cut(line))
            seg_list = pattern.findall(line)
            word_list = []
            for word in seg_list:
                if word not in stop_words:
                    word_list.append(word)
            if len(word_list) > 0:
                sentecnces.append(word_list)
                line = " ".join(word_list)
                f_writer.write(line + "\n")
                f_writer.flush()
    f_writer.close()
    return sentecnces

In [26]:
dir_current = os.getcwd()
# 注意此处的路径，在 jupyter_notebook_config.py 中设置，否则会报错
stop_words_file = os.path.join(dir_current, "learning_word-vector/data/stop_words.txt")
data_file = os.path.join(dir_current, "learning_word-vector/data/天龙八部.txt")
split_data_file = os.path.join(dir_current, "learning_word-vector/data/天龙八部_split.txt")

stop_words = read_stop(stop_words_file)
sentecnces = data_process(stop_words, data_file, split_data_file)
print(len(sentecnces))
sentecnces[:10]

10961


[['书名', '天龙八部'],
 ['作者', '金庸'],
 ['本文', '早安', '电子书', '网友', '分享', '版权', '原作者'],
 ['用于', '商业行为', '后果自负'],
 ['早安', '电子书'],
 ['金庸', '作品集', '三联', '版', '序'],
 ['小学',
  '时',
  '爱读',
  '课外书',
  '低年级',
  '时看',
  '儿童',
  '画报',
  '小朋友',
  '小学生',
  '内容',
  '小朋友',
  '文库',
  '似懂非懂',
  '阅读',
  '各种各样',
  '章回小说',
  '五六年',
  '级',
  '时',
  '看新',
  '文艺作品',
  '喜爱',
  '古典文学',
  '作品',
  '多于',
  '近代',
  '当代',
  '新文学',
  '个性',
  '使然',
  '朋友',
  '喜欢',
  '新文学',
  '不爱',
  '古典文学'],
 ['知识',
  '当代',
  '书报',
  '中',
  '寻求',
  '小学',
  '时代',
  '得益',
  '记忆',
  '最深',
  '爸爸',
  '哥哥',
  '购置',
  '邹韬奋',
  '所撰',
  '萍踪',
  '寄语',
  '萍踪',
  '忆语',
  '世界各地',
  '旅行',
  '记',
  '主编',
  '生活',
  '周报',
  '新',
  '旧',
  '童年时代',
  '深受',
  '邹先生',
  '生活',
  '书店',
  '之惠',
  '生活',
  '书店',
  '三联书店',
  '组成部分',
  '十多年',
  '前',
  '香港三联书店',
  '签',
  '合同',
  '中国',
  '大陆',
  '地区',
  '出版',
  '小说',
  '因事',
  '未果',
  '重',
  '行',
  '筹划',
  '三联书店',
  '独家',
  '出版',
  '中国',
  '大陆',
  '地区',
  '简体字',
  '感到',
  '欣慰',
  '回忆',
  '昔日',
  '心中',
  '充满',
  '温馨',
  '之

In [27]:
# 模型训练
start_time = time.time()
print("开始训练模型...")
SG = 1 # 0表示CBOW，1表示Skip-gram

model = word2vec.Word2Vec(sentecnces, vector_size=128, epochs=50, window=5, min_count=6,
                          workers=10, sg=0,
                          negative=5, hs=0, seed=42, compute_loss=True,
                          )
print("训练模型结束，耗时：", time.time() - start_time)
# 保存模型
if SG == 0:
    model_name = "天龙八部_word2vec_CBOW.bin"
else:
    model_name = "天龙八部_word2vec_Skip-gram.bin"

model_path = os.path.join(dir_current, f"learning_word-vector/models/{model_name}")
print("保存模型到：", model_path)

model.save(model_path)

开始训练模型...
训练模型结束，耗时： 9.789485692977905
保存模型到： E:\Pycharm_Data\Pycharm_Project\MyNLP_Project\NLP-Learning-Workshop\learning_word-vector/models/天龙八部_word2vec_Skip-gram.bin


In [28]:
# 加载模型
model2 = word2vec.Word2Vec.load(model_path)

# 选出10个与乔峰最相近的10个词
for e in model2.wv.most_similar(positive=["乔峰"], topn=10):
    print(e[0], e[1])

# 计算两个词语的相似度
sim_value = model.wv.similarity('乔峰', '萧峰')
print(sim_value)

# 计算两个集合的相似度
list1 = ['乔峰', '萧远山']
list2 = ['慕容复', '慕容博']
sim_value = model.wv.n_similarity(list1, list2)
print(sim_value)

乔大爷 0.44025424122810364
谭公 0.42235711216926575
徐长老 0.4182719886302948
宋长老 0.41528820991516113
白世镜 0.39926669001579285
赵钱孙 0.3771824836730957
真相 0.3664160370826721
鲍千灵 0.3563803434371948
陈长老 0.3414948880672455
向乔峰 0.3404667377471924
0.2193088
0.298908


## 学习资源
● [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)

● [word2vec Parameter Learning Explained](https://arxiv.org/abs/1411.2738)

● [可视化 word2vec](https://ronxin.github.io/wevi/)

● **[深入浅出理解word2vec模型 (理论与源码分析)](https://mp.weixin.qq.com/s/mb1MmW5SOXwt0WihXPVcVg)
● **[1W字，六十张图让小白也能搞懂Word2vec ](https://mp.weixin.qq.com/s/PTRx8JhjCkNLfBh-G2IJDA)
● [万物皆可Vector之语言模型：从N-Gram到NNLM、RNNLM](https://mp.weixin.qq.com/s/XiHzsjTK0TpQSjaOC6qFjw)

● [万物皆可Vector之Word2vec：2个模型、2个优化及实战使用](https://mp.weixin.qq.com/s/hoQXBo2r4WGgxIGtODZkFw)

● [【NLP修炼系列之词向量（二）】详解Word2Vec原理篇](https://mp.weixin.qq.com/s/krpBy8MeXX7twtCDEXsWhQ)


## 代码学习
● [word2vec(词嵌入)原理及代码实现](https://zhuanlan.zhihu.com/p/476920885)
● [word2vec-include-datapreprocess](https://github.com/ttb1534/word2vec-include-datapreprocess)

● [Word2vec](https://github.com/SeanLee97/nlp_learning/tree/master/word2vec)